In [3]:
from collections import Counter
import pandas as pd
import spacy
from spacy_langdetect import LanguageDetector
from spacy.language import Language
from collections import Counter
from pandarallel import pandarallel
pandarallel.initialize()

d:\Users\Leon Jovanovic\anaconda3\envs\opj\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
@Language.factory('language_detector')
def language_detector(nlp, name):
    return LanguageDetector()

In [3]:

data = pd.DataFrame()
for slice in range(0, 8000, 1000):
    data = pd.concat([data, pd.read_json(f'scraper_booking/hotels_reviews_data_{slice}_{slice + 1000}.json', orient='index')])
data.reset_index(drop=True, inplace=True)

In [4]:
data.hotel_review = data.hotel_review.apply(lambda x: x.strip().replace('\n', ' '))

In [5]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('language_detector', last=True)

In [7]:
%%time
data['language'] = data['hotel_review'].apply(lambda x: nlp(x)._.language['language'])

CPU times: total: 39min 32s
Wall time: 39min 53s


In [23]:
Counter(data_lang.language.to_list())

Counter({'hr': 78769, 'sl': 23162})

In [21]:
data_lang = data[(data.language == 'sl') | (data.language == 'hr')]
data_lang.reset_index(drop=True, inplace=True)
data_lang.to_json("reviews_lang.json", orient='index')

In [25]:
data.to_json("data/reviews_all_lang.json", orient='index')

In [26]:
li = [1,2,3,4,5,6,7,8]
li[-4:]

[5, 6, 7, 8]

In [34]:
data[['category','hotel_review']].loc[0]

category                                                   hotels
hotel_review    Ljubazni domaćini. Sve na dohvat ruke. Odlična...
Name: 0, dtype: object

## Shuffle data

In [36]:
from sklearn.utils import shuffle
df = pd.read_json('data/reviews_lang.json', orient='index')
df = shuffle(df, random_state=0)
df.reset_index(drop=True, inplace=True)
# Add 4 label columns
entities = ['amenities', 'location', 'cleanliness', 'staff']
for entity in entities:
    df[entity] = 'n/a'
# Split to two
df1 = df.iloc[:len(df)//2, :].reset_index(drop=True)
df2 = df.iloc[len(df)//2:, :].reset_index(drop=True)
# save progress to file
for idx, df in enumerate([df1, df2]):
    out = df.to_json(orient='index', indent=4, force_ascii=False)
    with open(f'data/annotated_reviews_lang_{idx + 1}.json', 'w', encoding='utf-8') as f:
        f.write(out)